adding the raw inference outputs to the cropped_mosaics

In [21]:
import rasterio
import numpy as np
from skimage.transform import resize

# Load the TIFF file
with rasterio.open('../../notebook/cropped_mosaic_no_nan/52VDQ_cropped_mosaic.tif') as src:
    # Read the existing bands
    bands = src.read()
    print(bands.shape)

    # Load the NumPy array
    array = np.load('done_inference_tiles/inference_ensemble_52VDQ.npy')
    data = array[0]
    print(data.shape)
    data = np.moveaxis(data, 2, 0)
    print(data.shape)
    resized_data = resize(data, bands.shape)
    print(resized_data.shape)

    # Check if the dimensions of the array match the TIFF file
    if bands.shape[1:] != resized_data.shape[1:]:
        raise ValueError("Array dimensions do not match the TIFF file")

    # Create a new array with the additional band
    new_bands = np.concatenate((bands, resized_data), axis=0)

    profile = src.profile
    profile['count'] = 4

    # Create a new TIFF file with the additional band
    with rasterio.open('inference_tifs/52VDQ_inference.tif', 'w', **profile) as dst:
        dst.write(new_bands)


(2, 3611, 3610)
(10980, 10980, 2)


only reading center predictions for every patch

In [2]:
import rasterio
import numpy as np
from skimage.transform import resize

# Load the TIFF file
with rasterio.open('../../notebook/cropped_mosaic_no_nan/52VDQ_cropped_mosaic.tif') as src:
    # Read the existing bands
    bands = src.read()
    print(bands.shape)

    # Load the NumPy array
    array = np.load('done_inference_tiles/inference_ensemble_52VDQ.npy')
    data = array[0]
    print(data.shape)
    data = np.moveaxis(data, 2, 0)

    data = data[:,7::30,7::30]

    print(data.shape)
    resized_data = resize(data, bands.shape)
    print(resized_data.shape)

    # Check if the dimensions of the array match the TIFF file
    if bands.shape[1:] != resized_data.shape[1:]:
        raise ValueError("Array dimensions do not match the TIFF file")

    # Create a new array with the additional band
    new_bands = np.concatenate((bands, resized_data), axis=0)

    profile = src.profile
    profile['count'] = 4
    
    '''
    band 1: icesat agbd
    band 2: icesat std
    band 3: predicted agbd
    band 4: ensemble std
    
    '''

    # Create a new TIFF file with the additional band
    with rasterio.open('inference_tifs/52VDQ_inference_downsampled.tif', 'w', **profile) as dst:
        dst.write(new_bands)


(2, 3611, 3610)
(10980, 10980, 2)
(2, 366, 366)
(2, 3611, 3610)


including the difference between icesat and predicted agbd

In [5]:
import rasterio
import numpy as np
from skimage.transform import resize

# Load the TIFF file
with rasterio.open('../../notebook/cropped_mosaic_no_nan/52VDQ_cropped_mosaic.tif') as src:
    # Read the existing bands
    bands = src.read()
    print(bands.shape)

    # Load the NumPy array
    array = np.load('done_inference_tiles/inference_ensemble_52VDQ.npy')
    data = array[0]
    print(data.shape)
    data = np.moveaxis(data, 2, 0)

    data = data[:,7::15,7::15]

    print(data.shape)
    resized_data = resize(data, bands.shape)
    print(resized_data.shape)

    # Check if the dimensions of the array match the TIFF file
    if bands.shape[1:] != resized_data.shape[1:]:
        raise ValueError("Array dimensions do not match the TIFF file")

    difference = np.where(bands[0] == -9999, 0 - resized_data[0], bands[0] - resized_data[0])
    difference = np.expand_dims(difference, axis=0)
    # Create a new array with the additional band
    new_bands = np.concatenate((bands, resized_data, difference), axis=0)

    profile = src.profile
    profile['count'] = 5
    
    '''
    band 1: icesat agbd
    band 2: icesat std
    band 3: predicted agbd
    band 4: ensemble std
    band 5: difference between icesat and predicted agbd
    
    '''

    # Create a new TIFF file with the additional band
    with rasterio.open('inference_tifs/52VDQ_inference_difference.tif', 'w', **profile) as dst:
        dst.write(new_bands)


(2, 3611, 3610)
(10980, 10980, 2)
(2, 732, 732)
(2, 3611, 3610)


In [4]:
import os
import rasterio
import numpy as np
from skimage.transform import resize

# Specify the directory containing the numpy arrays
numpy_dir = 'done_inference_tiles/low_res/'

# Get a list of all numpy files in the directory
numpy_files = [f for f in os.listdir(numpy_dir) if f.endswith('.npy')]

# Loop over each numpy file
for numpy_file in numpy_files:
    # Load the TIFF file
    tile = numpy_file[-9:-4]
    print(tile)
    with rasterio.open(f'../../notebook/cropped_mosaic_no_nan/{tile}_cropped_mosaic.tif') as src:
        # Read the existing bands
        bands = src.read()

        # Load the NumPy array
        array = np.load(os.path.join(numpy_dir, numpy_file))
        data = array[0]
        data = np.moveaxis(data, 2, 0)

        data = data[:,7::30,7::30]

        resized_data = resize(data, bands.shape)

        # Check if the dimensions of the array match the TIFF file
        if bands.shape[1:] != resized_data.shape[1:]:
            raise ValueError("Array dimensions do not match the TIFF file")

        difference = np.where(bands[0] == -9999, 0 - resized_data[0], bands[0] - resized_data[0])
        difference = np.expand_dims(difference, axis=0)
        # Create a new array with the additional band
        new_bands = np.concatenate((bands, resized_data, difference), axis=0)

        profile = src.profile
        profile['count'] = 5

        # Create a new TIFF file with the additional band
        output_filename = 'inference_tifs/' + tile + '_inference_difference.tif'
        with rasterio.open(output_filename, 'w', **profile) as dst:
            dst.write(new_bands)

52VDM
52VDN
52VDP
52VEM
52VEN
52VEP
52VEQ
